In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [ ]:
DATA_FOLDER = "./cedas2025_material/data/"

In [ ]:
df = pd.read_parquet(DATA_FOLDER + "chargecurves_train.parquet")
df.tail()

In [ ]:
df.describe()
df[(df['nominal_power']>50) & (df['nominal_power'] < 100)].tail(20)


In [ ]:
#df_sampled = df.iloc[:int(len(df) * 0.6)]  # Velger de første 60%
#df_sampled.to_csv("chargecurves_train.csv", index=False)

#df_sampled.to_csv("chargecurves_train.csv", index=False)

In [ ]:
# Konvertering av timestamp til datetime-format
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Fjerne rader med nan-values
df_clean = df.dropna()

# 1. Distribusjon av SOC
plt.figure(figsize=(8, 5))
plt.hist(df_clean['soc'], bins=30, edgecolor='black', alpha=0.7)
plt.xlabel("State of Charge (SOC) [%]")
plt.ylabel("Antall forekomster")
plt.title("Distribusjon av SOC (State of Charge)")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# 2. Distribusjon av Power
plt.figure(figsize=(8, 5))
plt.hist(df_clean['power'], bins=30, edgecolor='black', alpha=0.7)
plt.xlabel("Power [kW]")
plt.ylabel("Antall forekomster")
plt.title("Distribusjon av ladekraft (Power)")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# 3. Scatterplot SOC vs. Power
plt.figure(figsize=(8, 5))
plt.scatter(df_clean['soc'], df_clean['power'], alpha=0.5)
plt.xlabel("State of Charge (SOC) [%]")
plt.ylabel("Power [kW]")
plt.title("Sammenheng mellom SOC og effekt")
plt.grid(linestyle='--', alpha=0.7)
plt.show()

# 4. SOC over tid (for et utvalg av en ID for tydeligere plot)
sample_id = df_clean['id'].iloc[88]  # Velge en ID for eksempelvisning
df_sample = df_clean[df_clean['id'] == sample_id]

def utvikling_soc_og_power(df_sample, sample_id):
    plt.figure(figsize=(12, 6))
    
    # Plott SOC over tid
    plt.plot(df_sample['timestamp'], df_sample['soc'], marker='o', linestyle='-', label="SOC [%]", color='blue')
    
    # Plott Power over tid
    plt.plot(df_sample['timestamp'], df_sample['power'], marker='o', linestyle='-', label="Power [kW]", color='red')
    
    # Konfigurer aksene og legg til tittel
    plt.xlabel("Tid", fontsize=12)
    plt.ylabel("Verdi", fontsize=12)
    plt.title(f"Utvikling av SOC og Power over tid (Eksempel ID {sample_id})", fontsize=14)
    plt.xticks(rotation=45)
    plt.legend()
    plt.grid(linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

# Kall funksjonen for å visualisere for et spesifikt ID-eksempel
utvikling_soc_og_power(df_sample, sample_id)


# 6. Sammenligning av SOC, Power og Nominal Power
plt.figure(figsize=(8, 5))
plt.scatter(df_clean['soc'], df_clean['power'], label="Power [kW]", alpha=0.5)
plt.scatter(df_clean['soc'], df_clean['nominal_power'], label="Nominal Power [kW]", alpha=0.5)
plt.xlabel("State of Charge (SOC) [%]")
plt.ylabel("Power [kW]")
plt.title("Forhold mellom SOC, effekt og nominell effekt")
plt.legend()
plt.grid(linestyle='--', alpha=0.7)
plt.show()


In [ ]:
# 1. Effekt over tid (for et utvalg ID-er)
unique_ids = df_clean['id'].unique()[:5]  # Velger de første 5 ID-ene for visualisering
plt.figure(figsize=(12, 6))
for uid in unique_ids:
    df_subset = df_clean[df_clean['id'] == uid]
    plt.plot(df_subset['timestamp'], df_subset['power'], label=f'ID {uid}', alpha=0.7)
plt.xlabel("Tid")
plt.ylabel("Power [kW]")
plt.title("Effekt over tid for ulike ID-er")
plt.xticks(rotation=45)
plt.legend()
plt.grid(linestyle='--', alpha=0.7)
plt.show()

# 2. SOC-utvikling for flere ID-er
plt.figure(figsize=(12, 6))
for uid in unique_ids:
    df_subset = df_clean[df_clean['id'] == uid]
    plt.plot(df_subset['timestamp'], df_subset['soc'], label=f'ID {uid}', alpha=0.7)
plt.xlabel("Tid")
plt.ylabel("State of Charge (SOC) [%]")
plt.title("SOC-utvikling over tid for ulike ID-er")
plt.xticks(rotation=45)
plt.legend()
plt.grid(linestyle='--', alpha=0.7)
plt.show()

# 3. Korrelasjonsmatrise
correlation_matrix = df_clean[['soc', 'power', 'nominal_power']].corr()
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Korrelasjon mellom variabler")
plt.show()

# 4. Fordeling av nominell effekt (nominal_power)
plt.figure(figsize=(8, 5))
plt.hist(df_clean['nominal_power'], bins=30, edgecolor='black', alpha=0.7)
plt.xlabel("Nominal Power [kW]")
plt.ylabel("Antall forekomster")
plt.title("Distribusjon av nominell effekt")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# 5. Sammenligning av effekt og nominell effekt over SOC
plt.figure(figsize=(10, 6))
plt.scatter(df_clean['soc'], df_clean['power'], label="Power [kW]", alpha=0.5)
plt.scatter(df_clean['soc'], df_clean['nominal_power'], label="Nominal Power [kW]", alpha=0.5)
plt.xlabel("State of Charge (SOC) [%]")
plt.ylabel("Power [kW]")
plt.title("Sammenligning av Power og Nominal Power over SOC")
plt.legend()
plt.grid(linestyle='--', alpha=0.7)
plt.show()

In [ ]:
# Konverter timestamp til datetime-format og ekstraher tidselementer
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['weekday'] = df['timestamp'].dt.day_name()  # Navn på ukedag
df['hour'] = df['timestamp'].dt.hour          # Klokkeslett
df['month'] = df['timestamp'].dt.month_name() # Navn på måned
df['date'] = df['timestamp'].dt.date          # Dato

# Gruppér etter ukedag og tell antall målinger
weekday_activity = df.groupby('weekday').size().reindex(
    ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
)

# Visualiser ukedagaktivitet
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
plt.bar(weekday_activity.index, weekday_activity.values, color='skyblue')
plt.title("Aktivitet per ukedag", fontsize=14)
plt.xlabel("Ukedag", fontsize=12)
plt.ylabel("Antall aktiviteter", fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# Gruppér etter time og tell antall målinger
hour_activity = df.groupby('hour').size()

# Visualiser klokkeslettaktivitet
plt.figure(figsize=(10, 6))
plt.plot(hour_activity.index, hour_activity.values, marker='o', linestyle='-', color='blue')
plt.title("Aktivitet per klokkeslett", fontsize=14)
plt.xlabel("Klokkeslett", fontsize=12)
plt.ylabel("Antall aktiviteter", fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


# Gruppér etter måned og tell antall målinger
month_activity = df.groupby('month').size().reindex([
    'January', 'February', 'March', 'April', 'May', 'June', 
    'July', 'August', 'September', 'October', 'November', 'December'
])

# Visualiser månedsaktivitet
plt.figure(figsize=(10, 6))
plt.bar(month_activity.index, month_activity.values, color='lightgreen')
plt.title("Aktivitet per måned", fontsize=14)
plt.xlabel("Måned", fontsize=12)
plt.ylabel("Antall aktiviteter", fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


# Gruppér etter ukedag og klokkeslett
grouped_activity = df.groupby(['weekday', 'hour']).size().unstack(fill_value=0)

# Visualiser som heatmap
import seaborn as sns
plt.figure(figsize=(12, 8))
sns.heatmap(grouped_activity, cmap="Blues", annot=False)
plt.title("Aktivitet per ukedag og klokkeslett", fontsize=14)
plt.xlabel("Klokkeslett", fontsize=12)
plt.ylabel("Ukedag", fontsize=12)
plt.tight_layout()
plt.show()


In [ ]:
# Her ser vi at location 100 har noe potensielle feil. 

# Filtrer rader der power > 
outliers = df[df['power'] > df['nominal_power']]

# Gruppér etter location_id og tell forekomster
outlier_summary = outliers.groupby(['location_id','id']).size().reset_index(name='Antall outliers')

# Sorter resultatene for oversikt
outlier_summary = outlier_summary.sort_values(by='Antall outliers', ascending=False)

# Vis resultatene
print(outlier_summary)

# Velg topp N maskiner med flest outliers
top_n = 20
top_outliers = outlier_summary.head(top_n)

# Lag et forbedret stolpediagram
plt.figure(figsize=(10, 6))
plt.bar(top_outliers['id'].astype(str), top_outliers['Antall outliers'], color='steelblue')
plt.xlabel("Maskin-ID", fontsize=12)
plt.ylabel("Antall Outliers", fontsize=12)
plt.title("Topp 20 maskiner med flest outliers", fontsize=14)
plt.xticks(rotation=45, fontsize=10)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# TO DO:
#1. Kan du også plotte hvilke hverdager er det mest lading, hvilke klokkeslett, hvilke måneder osv? 

#2. Hvis power er > nominal_power ==> Sa sett power til nominal_power, for hvis det er det sa er det sikkert noe feil. 

In [ ]:
# Finn topp outlier-ID-er
top_outliers_ids = outlier_summary['id'].head(20).values  # Topp 20 maskin-ID-er

# Filtrer data for disse ID-ene
filtered_data = df[df['id'].isin(top_outliers_ids)]

# Finn topp outlier-ID-er
top_outliers_ids = outlier_summary['id'].head(20).values  # Topp 20 maskin-ID-er

# Filtrer data for disse ID-ene
filtered_data = df[df['id'].isin(top_outliers_ids)]

# Plott SOC vs Power for hver ID i et separat plott
for outlier_id in top_outliers_ids:
    id_data = filtered_data[filtered_data['id'] == outlier_id]
    
    plt.figure(figsize=(8, 5))
    plt.plot(id_data['soc'], id_data['power'], label=f"ID {outlier_id}", alpha=0.7)
    plt.xlabel("State of Charge (SOC) [%]", fontsize=12)
    plt.ylabel("Power [kW]", fontsize=12)
    plt.title(f"SOC vs Power for ID {outlier_id}", fontsize=14)
    plt.grid(linestyle='--', alpha=0.7)
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
# Finn topp outlier-ID-er basert på antall outliers
significant_outliers = outlier_summary[outlier_summary['Antall outliers'] > 5]  # Velg ID-er med mer enn 5 outliers
top_outliers_ids = significant_outliers['id'].values

# Filtrer data for disse ID-ene
filtered_data = df[df['id'].isin(top_outliers_ids)]

# Plott SOC vs Power med nominell effekt for hver ID i et separat plott
for outlier_id in top_outliers_ids:
    id_data = filtered_data[filtered_data['id'] == outlier_id]
    nominal_power = id_data['nominal_power'].iloc[0]
    
    plt.figure(figsize=(8, 5))
    plt.plot(id_data['soc'], id_data['power'], label=f"Power (ID {outlier_id})", alpha=0.7)
    plt.axhline(y=nominal_power, color='red', linestyle='--', label="Nominal Power", linewidth=2)
    plt.xlabel("State of Charge (SOC) [%]", fontsize=12)
    plt.ylabel("Power [kW]", fontsize=12)
    plt.title(f"SOC vs Power with Nominal Power for ID {outlier_id}", fontsize=14)
    plt.grid(linestyle='--', alpha=0.7)
    plt.legend()
    plt.tight_layout()
    plt.show()


In [ ]:
# Filtrer rader der power > nominal_power (outliers)
outliers = df[df['power'] > df['nominal_power']]

# Gruppér etter location_id og tell antall outliers
location_outliers = outliers.groupby('location_id').size().reset_index(name='Antall outliers')

# Sorter etter antall outliers
location_outliers = location_outliers.sort_values(by='Antall outliers', ascending=False)

# Vis de topp lokasjonene med potensielle problemer
print(location_outliers.head(10))

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

# Beregn gjennomsnittsverdier for relevante variabler per maskin (id)
agg_data = df.groupby('id').agg({
    'power': 'mean',        # Gjennomsnittlig ladekraft
    'soc': 'mean',          # Gjennomsnittlig SOC
    'nominal_power': 'mean' # Gjennomsnittlig nominell effekt
}).reset_index()

# Fyll inn manglende verdier med gjennomsnitt
agg_data.fillna(agg_data.mean(), inplace=True)

# Skaler data for clustering
scaler = StandardScaler()
scaled_data = scaler.fit_transform(agg_data[['power', 'soc', 'nominal_power']])

scaler = StandardScaler()
# Kjør K-Means clustering
kmeans = KMeans(n_clusters=7, random_state=42)
agg_data['cluster'] = kmeans.fit_predict(scaled_data)

# Visualiser clusterne
plt.figure(figsize=(10, 6))
sns.scatterplot(
    data=agg_data,
    x='power',
    y='soc',
    hue='cluster',
    palette="viridis",
    s=100
)
plt.title("Clustering av maskiner basert på gjennomsnittsverdier")
plt.xlabel("Gjennomsnittlig Power [kW]")
plt.ylabel("Gjennomsnittlig SOC [%]")
plt.legend(title="Cluster")
plt.grid(linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# Vis cluster-statistikk
cluster_summary = agg_data.groupby('cluster').mean()
print(cluster_summary)
